In [2]:
import pandas as pd    
import geopandas as gpd
from geopandas.tools import geocode
import numpy as np
import scipy.stats as stats
import scipy
import seaborn as sns
import matplotlib 
from matplotlib import pyplot as plt
matplotlib.rcParams.update({'font.size': 20})

In [3]:
import urllib.request as request

**Imports all traffic data from 2007 - 2018**

Link: https://data.seattle.gov/browse?q=traffic%20flow%20count&sortBy=relevance to download yearly data

In [68]:
url_list = ['https://opendata.arcgis.com/datasets/7015d5d46a284f94ac05c2ea4358bcd7_0.geojson',
            'https://opendata.arcgis.com/datasets/5fc63b2a48474100b560a7d98b5097d7_1.geojson',
            'https://opendata.arcgis.com/datasets/27af9a2485c5442bb061fa7e881d7022_2.geojson',
            'https://opendata.arcgis.com/datasets/4f62515558174f53979b3be0335004d3_3.geojson',
            'https://opendata.arcgis.com/datasets/29f801d03c9b4b608bca6a8e497278c3_4.geojson',
            'https://opendata.arcgis.com/datasets/a0019dd0d6464747a88921f5e103d509_5.geojson',
            'https://opendata.arcgis.com/datasets/40bcfbc4054549ebba8b5777bbdd40ff_6.geojson',
            'https://opendata.arcgis.com/datasets/16cedd233d914118a275c6510115d466_7.geojson',
            'https://opendata.arcgis.com/datasets/902fd604ecf54adf8579894508cacc68_8.geojson',
            'https://opendata.arcgis.com/datasets/170b764c52f34c9497720c0463f3b58b_9.geojson',
            'https://opendata.arcgis.com/datasets/2c37babc94d64bbb938a9b520bc5538c_10.geojson',
            'https://opendata.arcgis.com/datasets/a35aa9249110472ba2c69cc574eff984_11.geojson']
    

In [72]:
a = gpd.read_file(url_list[0])

#a = get_gdf(7)

a.head()

,OBJECTID,GEOBASID,DOWNTOWN,COMPKEY,STNAME,OID_,NAME,YEAR,SEGKEY,AAWDT,INPUT_STUD,HPMS,FLOW_COUNT,SHAPE_Length,geometry
0,1,94250370,N,12522,RENTON AVE S,135,"RENTON AVE S, N/O S CLOVERDALE ST",2007,12522,7200,0,,Y,4615.134504,"LINESTRING (-122.28064 47.53541, -122.28056 47..."
1,2,50000650,N,8702,AIRPORT WAY S,208,"AIRPORT WAY S, N/O S NORFOLK ST",2007,8702,13100,0,,Y,18205.478931,"MULTILINESTRING ((-122.29348 47.52408, -122.29..."
2,3,127150040,N,16114,N NORTHGATE WAY,56,"N NORTHGATE WAY, W/O ASHWORTH AVE N",2007,16114,28300,0,,Y,4267.857028,"LINESTRING (-122.34469 47.70505, -122.34334 47..."
3,4,123450090,N,15794,N 65TH ST,127,"N 65TH ST, W/O LINDEN AVE N",2007,15794,8200,0,,Y,12341.641545,"LINESTRING (-122.34831 47.67606, -122.34990 47..."
4,5,9550200,Y,2762,2ND AVE,145,"2ND AVE, NW/O LENORA ST",2007,2762,12400,0,,Y,4161.475805,"LINESTRING (-122.34045 47.61096, -122.34165 47..."


In [5]:
def get_gdf(year):
    '''Enter the desired year to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007.
    '''
    num = year-7
    gdf_year = gpd.read_file(url_list[num])
    if year == 11:
        gdf_year = gdf_year.rename(columns={"YEAR_" : 'YEAR'})
    if year == 12:
        gdf_year = gdf_year.rename(columns={'STDY_YEAR' : 'YEAR'})
    if year == 15 or year == 16:
        gdf_year = gdf_year.rename(columns={"COUNTAAWDT" : 'AAWDT', "FLOWSEGID" : 
                                            "GEOBASID", 'FIRST_STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 15:
            year_list = ['2015']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 16:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    elif year == 17 or year == 18:
        gdf_year = gdf_year.rename(columns={"AWDT" : 'AAWDT', "FLOWSEGID" : "GEOBASID", 
                                            'STNAME_ORD' : 'STNAME'})
        gdf_year = gdf_year[['AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 'geometry']]
        if year == 17:
            year_list = ['2017']*len(gdf_year)
            gdf_year['YEAR'] = year_list
        elif year == 18:
            year_list = ['2016']*len(gdf_year)
            gdf_year['YEAR'] = year_list
    #df_year_AAWDT = df_year['AAWDT'].values
    #df_year_geobase = df_year['GEOBASID'].values
    #df_year_dist = df_year['SHAPE_Length'].values
    gdf_year = gdf_year[[ 'YEAR', 'AAWDT', 'GEOBASID', 'STNAME', 'SHAPE_Length', 
                         'geometry']]
    return gdf_year #, df_year_AAWDT, df_year_geobase, df_year_dist

In [12]:
# Creates a dictionary of dataframes and a list of years
dfs = {}
years = np.arange(7,19,1)

for year in years:
    dfs[year] = get_gdf(year)

In [65]:
def get_st_data(year_list, stname_list):
    '''Enter a list of the desired years and street name (as a list of strings) to download the traffic flow count
    data for that year. Example: enter '7' for the year 2007. Make sure to enter a list even if only requesting a single 
    year or street name.'''
    all_data = gpd.GeoDataFrame()
    for i in range(len(year_list)):
        year = year_list[i]
        gdf_year = get_gdf(year)
        all_data = all_data.append(gdf_year)
    st_data_year = gpd.GeoDataFrame()
    for j in range(len(stname_list)):
        st_data_year = st_data_year.append(all_data.loc[all_data['STNAME'] == stname_list[j]])
    return st_data_year

In [66]:
montlake_br_data_08 = get_st_data([8, 12], ['MONTLAKE BLVD NE', 'MONTLAKE BR'])

In [67]:
montlake_br_data_08

,YEAR,AAWDT,GEOBASID,STNAME,SHAPE_Length,geometry
149,2008.0,46100.0,86950030.0,MONTLAKE BLVD NE,3751.859475,"LINESTRING (-122.30459 47.64900, -122.30458 47..."
108,2012.0,42312.0,86950030.0,MONTLAKE BLVD NE,2966.829559,"LINESTRING (-122.30391 47.65108, -122.30345 47..."
198,2008.0,57100.0,86850010.0,MONTLAKE BR,1362.081215,"LINESTRING (-122.30462 47.64526, -122.30461 47..."
140,2012.0,55764.0,86850010.0,MONTLAKE BR,2147.111130,"LINESTRING (-122.30462 47.64526, -122.30461 47..."
